# Semantop


In [ ]:
# Copyright 2025 Colin de Seroux alias Phenix333 (https://colindeseroux.fr)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## <span style="color:#10c0ff">Dependencies installation</span>


In [ ]:
%pip install -r ../requirements.txt

## <span style="color:#10c0ff">Dependencies importation</span>


In [ ]:
from gensim.models import KeyedVectors
import nltk
from nltk.corpus import stopwords
import os
from os import path as osp
import random
import requests
import zipfile

## <span style="color:#10c0ff">Environment</span>


In [ ]:
MODEL = "frWac_no_postag_phrase_500_cbow_cut10"
DICO_DELA = "http://infolingu.univ-mlv.fr/DonneesLinguistiques/Dictionnaires/dela-fr-public.zip?B1=T%E9l%E9charger"
DICO_HBENBEL = "https://raw.githubusercontent.com/hbenbel/French-Dictionary/refs/heads/master/dictionary"

## <span style="color:#10c0ff">Download models</span>


In [ ]:
os.makedirs("../models", exist_ok=True)

### <span style="color:#10ff41">Download embedding model</span>


In [ ]:
if osp.exists(f"../models/{MODEL}.bin"):
    print(f"File ../models/{MODEL}.bin already exists.")
else:
    with open(f"../models/{MODEL}.bin", "wb") as f:
        f.write(requests.get(f"https://embeddings.net/embeddings/{MODEL}.bin").content)
    
    print(f"File ../models/{MODEL} downloaded successfully.")

### <span style="color:#10ff41">Download correct word files</span>


In [ ]:
if osp.exists(f"../models/dico.dic"):
    print(f"File ../models/dico.dic already exists.")
else:
    with open("../models/dico.zip", "wb") as f:
        f.write(requests.get(DICO_DELA).content)

    with zipfile.ZipFile("../models/dico.zip", "r") as zip_ref:
        with zip_ref.open("dela-fr-public.dic") as source, open("../models/dico.dic", "wb") as target:
            target.write(source.read())
    
    print(f"File ../models/dico.dic downloaded successfully.")

In [ ]:
files = ["adj", "adv", "noun", "verb"]

if osp.exists(f"../models/dico.csv"):
    print(f"File ../models/dico.csv already exists.")
else:
    for f in files:
        response = requests.get(f"{DICO_HBENBEL}/{f}.csv")
        
        with open(f"../models/{f}.csv", "w", encoding="utf-8") as f:
            f.write(response.text)

## <span style="color:#10c0ff">Preprocessing the dico</span>


### <span style="color:#10ff41">Load correct_words if `dico.csv` exist</span>


In [ ]:
correct_words = []

if osp.exists(f"../models/dico.csv"):
    print(f"File ../models/dico.csv already exists.")
    
    with open("../models/dico.csv", "r", encoding="utf-8") as f:
        correct_words = [word for word in f]
    
    print(f"Loaded {len(correct_words)} words from dico.csv")

### <span style="color:#10ff41">Preprocessing `dico.dic` to keep only the base of words</span>


In [ ]:
if not osp.exists(f"../models/dico.csv"):
    nouns = []
    verbs = []
    adjectives = []

    with open("../models/dico.dic", "r", encoding="utf-16") as f:
        for line in f:
            try:
                # Strip whitespace and split on comma
                line = line.strip()
                
                # Extract word and its classification
                if ",.N" in line and "NPropre" not in line:
                    word = line.split(',')[0]
                    nouns.append(word)
                elif ",.V" in line:
                    word = line.split(',')[0]
                    verbs.append(word)
                elif ",.A" in line:
                    word = line.split(',')[0]
                    adjectives.append(word)
            except Exception as e:
                continue

    correct_words = nouns + verbs + adjectives

    print(f"Found {len(adjectives)} adjectives, {len(nouns)} nouns and {len(verbs)} verbs ")
    print(f"Total correct words in dico.dic: {len(correct_words)}")

Found 35650 adjectives, 128600 nouns and 12109 verbs  
Total correct words in dico.dic: 176359


### <span style="color:#10ff41">Preprocessing `adj.csv`, `adv.csv`, `noun.csv` and `verb.csv` to keep only the base of words</span>


In [ ]:
if not osp.exists(f"../models/dico.csv"):
    words = {
        "adj": 0,
        "adv": 0,
        "noun": 0,
        "verb": 0
    }
    
    for f_n in files:
        with open(f"../models/{f_n}.csv", "r", encoding="utf-8") as f:
            lines = f.readlines()
            
            matching_lines = []
            
            for line in lines:
                # print(line)
                line = line.strip()
                
                if line.endswith(",") or line.endswith(",['infinitive']"):
                    matching_lines.append(line.replace(",", "").replace("['infinitive']", ""))
            
            words[f_n] = len(matching_lines)
            
            correct_words.extend(matching_lines)
    
    print(f"Found {words['adj']} adjectives, {words['adv']} adverbs, {words['noun']} nouns and {words['verb']} verbs")
    print(f"Total correct words in adj.csv + adv.csv + noun.csv + verb.csv: {words['adj'] + words['adv'] + words['noun'] + words['verb']}")
    
    # Remove duplicates from the correct_words list to ensure uniqueness
    correct_words = list(set(correct_words))
            
    print(f"Found {len(correct_words)} correct words")

Found 16338 adjectives, 4229 adverbs, 43102 nouns and 14095 verbs  
Total correct words in adj.csv + adv.csv + noun.csv + verb.csv: 77764  
Found 191233 correct words


### <span style="color:#10ff41">Save correct_words in `dico.csv` if `dico.csv` do not exist</span>


In [ ]:
if not osp.exists(f"../models/dico.csv"):
    with open("../models/dico.csv", "w", encoding="utf-8") as f:      
        for word in correct_words:
            f.write(word + "\n")

## <span style="color:#10c0ff">Create the custom model</span>


### <span style="color:#10ff41">Load the model</span>


In [ ]:
model = KeyedVectors.load_word2vec_format(f"../models/{MODEL}.bin", binary=True, unicode_errors="ignore")

### <span style="color:#10ff41">Download / load french stopwords from nltk</span>


In [ ]:
nltk.download("stopwords")
french_stopwords = set(stopwords.words("french"))

### <span style="color:#10ff41">Delete stopwords</span>


In [ ]:
filtered_keys_without_stopwords = [word for word in model.index_to_key if word not in french_stopwords]
print(f"Filtered keys without stopwords: {len(filtered_keys_without_stopwords)}")

Filtered keys without stopwords: 1081851


### <span style="color:#10ff41">Keep only correct words if they exist in the model</span>


In [ ]:
correct_filtered_keys = [word for word in correct_words if word in filtered_keys_without_stopwords]
print(f"Correct filtered keys: {len(correct_filtered_keys)}")

Correct filtered keys: 61775


Just in case you re-launch the creation of a new model without paying attention.


In [ ]:
if not osp.exists(f"../models/correct_filtered_keys.csv"):
    with open("../models/correct_filtered_keys.csv", "w", encoding="utf-8") as f:      
        for word in correct_filtered_keys:
            f.write(word + "\n")

### <span style="color:#10ff41">Save the model</span>


In [ ]:
custom_model = KeyedVectors(vector_size=model.vector_size)

In [ ]:
custom_model.add_vectors(correct_filtered_keys, [model[word] for word in correct_filtered_keys])

In [ ]:
custom_model.save_word2vec_format(f"../models/{MODEL}_custom.bin", binary=True)

## <span style="color:#10c0ff">Test the custom model</span>


You can download directely the custom model on [HuggingFace](https://huggingface.co/datasets/colindeseroux/semantop).


### <span style="color:#10ff41">Load the model</span>


In [ ]:
custom_model = KeyedVectors.load_word2vec_format(f"../models/{MODEL}_custom.bin", binary=True, unicode_errors="ignore")

### <span style="color:#10ff41">Get random word</span>


In [ ]:
random_word = random.choice(list(custom_model.index_to_key))
print(f"Random word from the model: {random_word}")

### <span style="color:#10ff41">Get similar words</span>


In [ ]:
similar_words = custom_model.similar_by_word(random_word, topn=10)

print("Most similar words:")

for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

### <span style="color:#10ff41">Test word</span>


In [ ]:
similarity = custom_model.similarity(random_word, "chat")
print(f"Similarity between '{random_word}' and 'chat': {similarity:.4f}")

### <span style="color:#10ff41">Bot</span>


Test the model with assert in reverse.


In [ ]:
TARGETS = {
    "chat": -0.0195,
    "minou": 0.0167
}


def distance_scores(word: str, vectors: list, targets: list, model: KeyedVectors) -> float:
    """
    Calculate the cumulative squared error between the similarity of a word and target values.
    
    :param word: The word to evaluate.
    :type word: str
    :param vectors: List of reference words to compare against.
    :type vectors: list
    :param targets: List of target similarity values corresponding to the reference words.
    :type targets: list
    :param model: The word embedding model.
    :type model: KeyedVectors
    
    :return: Cumulative squared error.
    :rtype: float
    """
    
    if word not in model:
        return float("inf")
    
    score = 0.0
    
    for word_ref, target in zip(vectors, targets):
        sim = model.similarity(word, word_ref)
        score += (sim - target) ** 2
        
    return score

ref_words = list(TARGETS.keys())
target_values = list(TARGETS.values())

scores = []

for word in custom_model.index_to_key:
    if word in ref_words:
        continue

    dist = distance_scores(word, ref_words, target_values, custom_model)
    scores.append((word, dist))

scores = sorted(scores, key=lambda x: x[1])

assert scores[0][0] == "colin" # If not True, the model is not working as expected

print("Words closest to the combination :")

for word, error in scores[:10]:
    print(f"{word} (cumulative squared error: {error:.4f})")